# Semantic Similarity Example

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use real OpenAI or Hegel AI API keys, so for now we'll set them to empty strings.

In [1]:
import os
os.environ['DEBUG']="1"
os.environ['HEGELAI_API_KEY'] = ""
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [2]:
from typing import Dict, Tuple
from prompttools.harness.prompt_template_harness import (
    PromptTemplateExperimentationHarness,
)

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [3]:
prompt_templates = ["Answer the following question: {{input}}", "Respond the following query: {{input}}"]
user_inputs = [{"input": "Who was the first president?"}, {"input": "Who was the first president of India?"}]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [4]:
harness = PromptTemplateExperimentationHarness("text-davinci-003", prompt_templates, user_inputs)

We can then run the experiment to get results.

In [5]:
harness.prepare()
harness.run()
harness.visualize()

,messages,response(s),latency
0,Answer the following question: Who was the first president?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000004
1,Answer the following question: Who was the first president of India?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000002
2,Respond the following query: Who was the first president?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000002
3,Respond the following query: Who was the first president of India?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000001


You can use the `pivot` keyword argument to view results by the template and inputs that created them.

In [6]:
harness.visualize(pivot=True)

prompt_template,Answer the following question: {{input}},Respond the following query: {{input}}
user_input,,
{'input': 'Who was the first president of India?'},[\n\nThe Los Angeles Dodgers won the World Series in 2020],[\n\nThe Los Angeles Dodgers won the World Series in 2020]
{'input': 'Who was the first president?'},[\n\nThe Los Angeles Dodgers won the World Series in 2020],[\n\nThe Los Angeles Dodgers won the World Series in 2020]


## Evaluate the model response

To evaluate the results, we'll define an eval function. Since we are prompting the model to echo our input, we can use semantic distance to check if the model's response is similar to the user input.

In [7]:
from prompttools.utils import similarity


EXPECTED = {"Who was the first president?": "George Washington",
            "Who was the first president of India?": "Rajendra Prasad"}

def extract_responses(output) -> str:
    return [choice["text"] for choice in output["choices"]]


def measure_similarity(
    input_pair: Tuple[str, Dict[str, str]], results: Dict, metadata: Dict
) -> float:
    """
    A simple test that checks semantic similarity between the user input
    and the model's text responses.
    """
    distances = [
        similarity.compute(EXPECTED[input_pair[1]["input"]], response)
        for response in extract_responses(results)
    ]
    return min(distances)
    


Let's test our similarity function.

In [8]:
measure_similarity((prompt_templates[0], user_inputs[0]), {"choices": [{"text": "This is a test"}, {"text": "This is a text"}]}, {})

[2023-07-05 09:19:54,457] INFO in posthog: Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
[2023-07-05 09:19:54,471] INFO in ctypes: Successfully imported ClickHouse Connect C data optimizations
[2023-07-05 09:19:54,472] INFO in ctypes: Successfully import ClickHouse Connect C/Numpy optimizations
[2023-07-05 09:19:54,478] INFO in json_impl: Using python library for writing JSON byte strings
[2023-07-05 09:19:54,558] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[2023-07-05 09:19:54,844] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


0.8258005976676941

Finally, we can evaluate and visualize the results.

In [9]:
harness.evaluate("similar_to_expected", measure_similarity)
harness.visualize()

[2023-07-05 09:19:55,087] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[2023-07-05 09:19:55,320] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[2023-07-05 09:19:55,552] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[2023-07-05 09:19:55,787] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


,messages,response(s),latency,similar_to_expected
0,Answer the following question: Who was the first president?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000004,0.965657
1,Answer the following question: Who was the first president of India?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000002,1.021784
2,Respond the following query: Who was the first president?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000002,0.965657
3,Respond the following query: Who was the first president of India?,[\n\nThe Los Angeles Dodgers won the World Series in 2020],0.000001,1.021784
